In [6]:
import torch
import torch.nn as nn
import seaborn as sns
import torch.nn.functional as F
from torch.utils.data import  DataLoader
import numpy as np

In [2]:
cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda" if cuda else "cpu")

batch_size = 64

x_dim  = 2*20*10
hidden_dim = 100
latent_dim = 32

lr = 1e-3
epochs = 30
DEVICE

device(type='cuda')

## Load data

In [2]:
train_dataset = torch.load('nyc_grid_shifted_train_dataset.pt')
test_dataset = torch.load('nyc_grid_shifted_test_dataset.pt')

train_dataset[0]

(tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 1, 2, 6, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 1, 3, 2, 5, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 2, 9, 0, 2, 0, 1, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 1, 1, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 
         [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 3, 1, 2, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 2, 2, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 1, 2, 3, 2, 1, 1, 2, 4, 1, 0, 0, 0, 0, 0, 0,

In [3]:
len(train_dataset)

3896

In [13]:
train_dataset = torch.load('nyc_grid_shifted_train_dataset.pt')
test_dataset = torch.load('nyc_grid_shifted_test_dataset.pt')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers = 4)

In [4]:
x,y = next(iter(train_loader))
x.shape

torch.Size([64, 2, 10, 20])

### Encoder

In [5]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def forward(self, x):
        h_       = self.LeakyReLU(self.FC_input(x))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_)                     
        
        return mean, log_var

In [9]:
class Encoder2(nn.Module):
    
    def __init__(self, latent_dim, input_channels):
        super(Encoder2, self).__init__()
        
        self.latent_dim = latent_dim

        modules = []
        hidden_dims = [32, 64, 128, 256, 512,1000]
        in_channels = input_channels

        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels=in_channels, out_channels=h_dim,
                              kernel_size= 3, stride= 2, padding  = 1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
#         self.fc_mu = nn.Linear(hidden_dims[-1]*4, latent_dim)
#         self.fc_var = nn.Linear(hidden_dims[-1]*4, latent_dim)
        self.fc_mu = nn.Linear(hidden_dims[-1], latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1], latent_dim)
        
        self.training = True
        
    def forward(self, x):
        result = self.encoder(x)
        result = torch.flatten(result, start_dim=1)

        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return mu, log_var

### Decoder

In [6]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        h     = self.LeakyReLU(self.FC_hidden(x))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        
        x_hat = torch.sigmoid(self.FC_output(h))
        return x_hat

In [11]:
class Decoder2(nn.Module):
    def __init__(self, latent_dim, batch_size):
        super(Decoder2, self).__init__()
        
        hidden_dims = [32, 64, 128, 256, 512]
        hidden_dims.reverse()
        
#         self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1]*10*10)
        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1])
        
        modules = []
        
        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= 2,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())
        
    def forward(self, x):
        print('Input for decoder: ', x.shape)
        result = self.decoder_input(x)
        print('Decoder_input:', result.shape)
#         result = result.view(-1, 512, 2, 2)
#         result = result.view(-1, 2, 10, 10)
        print('Results shape:', result.shape)
        result = self.decoder(result)
        result = self.final_layer(result)

        return result

### Together

In [7]:
class VAE(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(VAE, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
        
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(DEVICE)        # sampling epsilon        
        z = mean + var*epsilon                          # reparameterization trick
        return z
                  
    def forward(self, x):
        mean, log_var = self.Encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var)
        x_hat = self.Decoder(z)
        
        return x_hat, mean, log_var

## Model declaration

In [8]:
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
# encoder2 = Encoder2(latent_dim=20, input_channels=2)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)
# decoder2 = Decoder2(latent_dim=20, batch_size=batch_size)

autoencoder = VAE(Encoder=encoder, Decoder=decoder).to(DEVICE)

## Loss function and optimizer

In [9]:
from torch.optim import Adam

BCE_loss = nn.BCELoss()

def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD      = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD


optimizer = Adam(autoencoder.parameters(), lr=lr)

## Training

In [14]:
print("Start training VAE...")
autoencoder.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, (x, _) in enumerate(train_loader):
        print(x.shape)
        x = x.view(batch_size, x_dim)
        x = x.to(DEVICE)
        x = x.float()

        optimizer.zero_grad()

        x_hat, mean, log_var = autoencoder(x)
        loss = loss_function(x, x_hat, mean, log_var)
        
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    if epoch % 5 == 0:
        print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))
    
print("Finish!")

Start training VAE...
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20])
torch.Size([64, 2, 10, 20]

RuntimeError: shape '[64, 400]' is invalid for input of size 22400

# Grid NN

#### Parameters

In [2]:
cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda" if cuda else "cpu")

batch_size = 256

lr = 1e-3
epochs = 30
DEVICE

device(type='cuda')

#### Network

In [3]:
class Net(nn.Module):
    def __init__(self, padding=0):
        super().__init__()
        self.padding = padding
        self.conv1 = nn.Conv2d(2, 8, 4, padding=self.padding, stride=2)
        self.conv2 = nn.Conv2d(8, 32, 3, padding=self.padding, stride=1)
        self.conv1_drop = nn.Dropout2d()
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(448, 128)
        self.fc2 = nn.Linear(128, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.conv1_drop(self.conv1(x)))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#### Load data

In [4]:
train_dataset = torch.load('nyc_grid_shifted_train_dataset.pt')
test_dataset = torch.load('nyc_grid_shifted_test_dataset.pt')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True, num_workers = 4)

x,y = next(iter(train_loader))
x.shape

torch.Size([256, 2, 10, 20])

#### Optimizer & criterion

In [5]:
net = Net().to(DEVICE)

criterion = nn.MSELoss().to(DEVICE)
optimizer = torch.optim.Adam(net.parameters(), lr)

In [ ]:
train_loss = []
test_loss = []

for epoch in range(epochs):
    epoch_losses_train = []
    epoch_losses_test = []
    
    for i, data in enumerate(train_loader):
        inputs, kpis = data
        
        inputs = inputs.float().to(DEVICE)
        kpis = kpis.to(DEVICE)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, kpis)
        loss.backward()
        optimizer.step()

        epoch_losses_train.append(loss.item())

        x, y = next(iter(test_loader))
        x = x.float().to(DEVICE)
        y = y.to(DEVICE)

        loss = criterion(y, net(x))
        epoch_losses_test.append(loss.item())

    train_loss.append(np.mean(epoch_losses_train))
    test_loss.append(np.mean(epoch_losses_test))
    if epoch % 10 == 9:
        print(f' ####### Epoch: {epoch} #######')
        print(f'Train loss: {train_loss[epoch]:.4f}')
        print(f'Test loss: {test_loss[epoch]:.4f}')
        print('\n')
plt.plot(train_loss, label='Train loss')
plt.plot(test_loss, label='Test loss')
plt.legend(title='Loss with respect to epoch')
plt.show()